In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metrics
import os
import tempfile 


from sklearn import preprocessing
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, classification_report
from collections import Counter

from imblearn.over_sampling import SMOTE

import tensorflow as tf
import keras


import warnings
warnings.filterwarnings("ignore")
colors = np.array(['#0101DF','#DF0101','#01DF01','#DFDFDF'])

rs = 42
def compute_metrics (y_te2,y_p,string):
    print('\n %s'%string)
    print('CONFUSION MATRIX')
    print(metrics.confusion_matrix(y_te2,y_p))
    print(classification_report(y_te2, y_p))
    print('\nROC CURVE: %2.2f'%roc_auc_score(y_te2,y_p))

def Ratio10(y,string):
    suma = Counter(y)[0]+Counter(y)[1]
    print('\n'+string)  
    print(Counter(y))
    print('0: %3.2f%%\n1: %3.2f%%' % (100*Counter(y)[0]/suma,100*Counter(y)[1]/suma))

def histo(data,var):
    fig=plt.figure()
    sns.countplot(var,data=data,palette=colors)
    plt.title('0/1 Distribution (0: No Stalling || 1: Stalling Event',fontsize=14)
    plt.show()
    
def plot_loss(history, label, n):
    # Use a log scale to show the wide range of values.
    plt.semilogy(history.epoch,  
                 history.history['loss'],
                 color=colors[n], 
                 label='Train '+label
                 )
    plt.semilogy(history.epoch,
                 history.history['val_loss'],
                 color=colors[n], 
                 label='Val '+label,
                 linestyle="--")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

def plot_metrics(history):
    metrics =  ['loss', 'auc', 'precision', 'recall']
    for n, metric in enumerate(metrics):
        name = metric.replace("_"," ").capitalize()
        plt.subplot(2,2,n+1)
        plt.plot(history.epoch,  history.history[metric], color=colors[0], label='Train')
        plt.plot(history.epoch, history.history['val_'+metric],
             color=colors[0], linestyle="--", label='Val')
        plt.xlabel('Epoch')
        plt.ylabel(name)
        if metric == 'loss':
            plt.ylim([0, plt.ylim()[1]])
        elif metric == 'auc':
            plt.ylim([0.5,1])
        else:
            plt.ylim([0,1])

    plt.legend()

def plot_cm(labels, predictions, p=0.5):
  cm = metrics.confusion_matrix(labels, predictions > p)
  plt.figure(figsize=(5,5))
  sns.heatmap(cm, annot=True, fmt="d")
  plt.title('Confusion matrix @{:.2f}'.format(p))
  plt.ylabel('Actual label')
  plt.xlabel('Predicted label')

  print('Legitimate Transactions Detected (True Negatives): ', cm[0][0])
  print('Legitimate Transactions Incorrectly Detected (False Positives): ', cm[0][1])
  print('Fraudulent Transactions Missed (False Negatives): ', cm[1][0])
  print('Fraudulent Transactions Detected (True Positives): ', cm[1][1])
  print('Total Fraudulent Transactions: ', np.sum(cm[1]))
  

def plot_roc(name, labels, predictions, **kwargs):
    fp, tp, _ = metrics.roc_curve(labels, predictions)

    plt.plot(100*fp, 100*tp, label=name, linewidth=2, **kwargs)
    plt.xlabel('False positives [%]')
    plt.ylabel('True positives [%]')
    plt.xlim([-0.5,100.5])
    plt.ylim([-0.5,100.5])
    plt.grid(True)
    ax = plt.gca()
    ax.set_aspect('equal')
  

In [ ]:
dataset = pd.read_csv('../../output/df_general.csv', sep=',')
split = 0.8
del dataset['No.']
del dataset['Protocol']
del dataset['ip_len']
del dataset['prebuffering']
del dataset['Time']

dataset = dataset.fillna(dataset.mean())
neg, pos = np.bincount(dataset['stalling_event'])
total = neg + pos
weight_for_0 = (1 / neg)*(total)/2.0 
weight_for_1 = (1 / pos)*(total)/2.0

class_weight = {0: weight_for_0, 1: weight_for_1}

print(dataset.columns)
Ratio10(dataset['stalling_event'],'dataset[stalling_event]')
histo(dataset,'stalling_event')

In [ ]:
from random import randint
'''X_train = [None] * (len(dataset.columns)-1)
X_test = [None] * (len(dataset.columns)-1)'''

X_train = np.array([None]*12)
X_test = np.array([None]*12)
y_train = list()
y_test = list()

grouped = dataset.groupby(['delay','capture']) 
#print(grouped.mean())

for id, data in grouped:
    spl = int(data.shape[0] * split)
    restspl = int(data.shape[0] * (1-split))
    randnum = randint(0,restspl)
    
    X_train = np.vstack((X_train,data.values[randnum:randnum+spl,:-3]))
    X_test = np.vstack((X_test,data.values[0:randnum-1,:-3]))
    X_test = np.vstack((X_test,data.values[randnum+spl:,:-3]))
    
    y_train.extend(data.values[randnum:randnum+spl,-1])
    y_test.extend(data.values[0:randnum-1,-1])
    y_test.extend(data.values[randnum+spl:,-1])

X_train = X_train[1:]
X_test = X_test[1:]

In [ ]:
mms = preprocessing.MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

In [ ]:
y_test = list(map(int,y_test))
y_train = list(map(int,y_train))
Ratio10(y_test,'y_test')
Ratio10(y_train,'y_train')

X_train = pd.DataFrame(data=X_train,columns=dataset.columns[:-3])
X_test= pd.DataFrame(data=X_test,columns=dataset.columns[:-3])
y_train = pd.DataFrame(data=y_train,columns=['y_train'])
y_test= pd.DataFrame(data=y_test,columns=['y_test'])
del dataset['delay']
del dataset['capture']

In [ ]:
smote = SMOTE(random_state = rs)
X_train_smote,y_train_smote = smote.fit_resample(X_train.values,y_train.values)
X_test_smote,y_test_smote = smote.fit_resample(X_test.values,y_test.values)

In [ ]:
EPOCHS = 100
BATCH_SIZE = 2048

resampled_steps_per_epoch = np.ceil(2.0*neg/BATCH_SIZE)
print(resampled_steps_per_epoch)

In [ ]:
METRICS = [
    tf.keras.metrics.TruePositives(name='tp'),
    tf.keras.metrics.FalsePositives(name='fp'),
    tf.keras.metrics.TrueNegatives(name='tn'),
    tf.keras.metrics.FalseNegatives(name='fn'),
    tf.keras.metrics.BinaryAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
]


early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', 
                                                  verbose=1,
                                                  patience=10,
                                                  mode='max',
                                                  restore_best_weights=True)



In [ ]:
n_features = X_train.shape[1]

initial_bias = np.log([pos/neg])
print(initial_bias)

In [ ]:

resampled_model = keras.Sequential()
resampled_model.add(keras.layers.Dense(1024,activation='sigmoid',input_shape=(n_features,)))
resampled_model.add(keras.layers.Dropout(0.5))
resampled_model.add(keras.layers.Dense(1024,activation='sigmoid'))
resampled_model.add(keras.layers.Dropout(0.5))
resampled_model.add(keras.layers.Dense(1024,activation='sigmoid'))
resampled_model.add(keras.layers.Dropout(0.5))
resampled_model.add(keras.layers.Dense(1,activation='sigmoid'))
print('Built NN...')
resampled_model.compile(optimizer=keras.optimizers.Adam(lr=1e-3),loss=keras.losses.BinaryCrossentropy(),metrics=METRICS)
resampled_model.summary()
print('Compiled NN...')

initial_weights = os.path.join(tempfile.mkdtemp(),'initial_weights')
resampled_model.save_weights(initial_weights)

resampled_model.load_weights(initial_weights)
output_layer = resampled_model.layers[-1] 
output_layer.bias.assign([0])

resampled_history = resampled_model.fit(X_train_smote,
                                        y_train_smote,
                                        batch_size=BATCH_SIZE,
                                        epochs=EPOCHS,
                                        callbacks = [early_stopping],
                                        validation_split=0.2
                                        )

In [ ]:
plot_metrics(resampled_history)
plt.show()

In [ ]:
train_predictions_resampled = resampled_model.predict(X_train_smote, batch_size=BATCH_SIZE)
test_predictions_resampled = resampled_model.predict(X_test_smote, batch_size=BATCH_SIZE)

In [ ]:
resampled_results = resampled_model.evaluate(X_test_smote,
                                             y_test_smote,
                                             batch_size=BATCH_SIZE, 
                                             verbose=1)

In [ ]:
for name, value in zip(resampled_model.metrics_names, resampled_results):
  print(name, ': ', value)
print()

plot_cm(y_test_smote, test_predictions_resampled)

In [ ]:





plot_roc("Train Resampled Smote", y_train_smote, train_predictions_resampled,  color=colors[2])
plot_roc("Test Resampled Smote", y_test_smote, test_predictions_resampled,  color=colors[2], linestyle='--')

plt.legend(loc='lower right')
